# Module 2, Section 2: Eval-Driven Development

The benefit of building out a dataset and eval metrics upfront is that we have a quantitative signal that we can use to understand how changes to our app impact downstream peformance. Without this, we're just flying blind!


<div align="center">
    <img src="../../images/eval_process.png">
</div>

### Systematically Improving Our Application

Our baseline evaluation revealed weaknesses in the current database sub-agent:

**The Problem:**
- ❌ **Excessive tool calls**: Sequential chains of `get_customer_orders` → multiple `get_order_status` → multiple `get_order_item_price` calls, etc.
- ❌ **Missing capabilities**: Can't aggregate (SUM, COUNT, AVG), filter (WHERE), or JOIN across tables - so relies on in memory math
- ❌ **Limited flexibility**: Rigid tools can't adapt to complex or novel queries

**Possible Solution:**
Build a flexible SQL agent that generates custom queries at runtime to answer any database question efficiently.

Let's use our evaluation setup to drive a targeted improvement!

## 1. Setup

We'll build a SQL agent that can generate custom queries to answer complex database questions.

<div align="center">
    <img src="../../images/db_agent_improvement.png">
</div>

In [ ]:
from dotenv import load_dotenv
from tools.database import get_database

load_dotenv()

# Get database connection (lazy loaded)
db = get_database()

# Extract schema once - we'll inject this into the agent's system prompt
table_info = db.get_table_info()

## 2. Build SQL Agent

Create a tool that executes SQL queries and an agent that can generate those queries on-demand using `create_agent()` abstraction.


In [ ]:
from langchain.tools import tool


@tool
def execute_sql(query: str) -> str:
    """Execute a SELECT query against the TechHub database.

    Safety: Only SELECT queries allowed - no INSERT/UPDATE/DELETE/etc.
    """
    # Safety check: Only allow SELECT queries
    if not query.strip().upper().startswith("SELECT"):
        return "Error: Only SELECT queries are allowed."

    # Block dangerous keywords
    FORBIDDEN = [
        "INSERT",
        "UPDATE",
        "DELETE",
        "ALTER",
        "DROP",
        "CREATE",
        "REPLACE",
        "TRUNCATE",
    ]
    if any(keyword in query.upper() for keyword in FORBIDDEN):
        return "Error: Query contains forbidden keyword."

    # Execute query
    db = get_database()  # lazy loaded
    try:
        result = db._execute(query)
        result = [tuple(row.values()) for row in result]  # extract values
        return result
    except Exception as e:
        return f"SQL Error: {str(e)}"

In [ ]:
SQL_AGENT_SYSTEM_PROMPT = f"""You are a database specialist for TechHub customer support.

You have access to a SQLite database with the following schema:

{table_info}

Your capabilities:
- Write SQL SELECT queries to answer any database question
- Use JOINs, aggregations (SUM, COUNT, AVG), filtering (WHERE), GROUP BY, ORDER BY
- Handle complex queries with multiple conditions

Guidelines:
1. Only use SELECT queries (read-only access)
2. Use proper JOINs when querying related tables
3. Format currency as $X.XX in your final answer
4. Provide context, not just raw numbers
5. If a query returns no results, explain why

Important: Read-only access - no INSERT/UPDATE/DELETE operations."""

In [ ]:
from langchain.agents import create_agent
from config import DEFAULT_MODEL

sql_agent = create_agent(
    model=DEFAULT_MODEL,
    tools=[execute_sql],
    name="sql_agent",
    system_prompt=SQL_AGENT_SYSTEM_PROMPT,
)
sql_agent

#### 🔒 Production Security Note

In this workshop, we're using keyword filtering to prevent write operations. In production PostgreSQL systems, you should implement defense-in-depth:

1. **Database-level**: Create a read-only user
2. **Connection-level**: Use SQLAlchemy's `postgresql_readonly=True` for example
3. **Application-level**: Validate queries before execution (our current approach)

SQLite doesn't support user roles, so our keyword approach is both educational and functional for this demo.

## 3. Quick Demo

Let's test the SQL agent on a query that previously failed in our baseline evaluation.


In [ ]:
# Test on a query that requires aggregation
question = "What items were in order ORD-2023-0002? How much did each cost?"

result = sql_agent.invoke(
    {"messages": [{"role": "user", "content": question}]},
)

for message in result["messages"]:
    message.pretty_print()

✅ **Success!** The SQL agent generates a custom query with JOINs and aggregation to answer the question efficiently in just a single tool call.

Check the LangSmith trace to see the generated SQL query!


## 4. Integrate SQL Agent with Supervisor HITL

Now let's swap our old rigid DB agent with this flexible SQL agent in the full supervisor HITL system.


In [ ]:
from agents.docs_agent import create_docs_agent
from agents.sql_agent import create_sql_agent
from agents.supervisor_hitl_agent import create_supervisor_hitl_agent

# Instantiate improved SQL agent for deployment
sql_agent = create_sql_agent()

# Instantiate docs agent
docs_agent = create_docs_agent()


# Compose supervisor HITL with SQL agent instead of old db_agent
improved_agent = create_supervisor_hitl_agent(
    db_agent=sql_agent,
    docs_agent=docs_agent,
)

In [ ]:
from IPython.display import Image

display(Image(improved_agent.get_graph(xray=True).draw_mermaid_png()))

Quick test:

In [ ]:
import uuid
from langgraph.types import Command

# New thread
thread_id = uuid.uuid4()
config = {"configurable": {"thread_id": thread_id}}

# First invocation - will pause at interrupt
result = improved_agent.invoke(
    {"messages": [{"role": "user", "content": "Whats the status of my recent order?"}]},
    config=config,
)

result

In [ ]:
# Resume with valid email
result = improved_agent.invoke(
    Command(resume="Ok, its: sarah.chen@gmail.com"),
    config=config,
)
result

In [ ]:
for message in result["messages"]:
    message.pretty_print()

## 5. Re-evaluate Performance

Now let's run the same evaluation on the same dataset to quantitatively see how it impacts our performance metrics.


In [ ]:
# Import evaluators from Section 1 (now packaged in evaluators module)
from evaluators import correctness_evaluator, count_total_tool_calls_evaluator

In [ ]:
import uuid
from langsmith import Client

client = Client()


def improved_target_function(inputs: dict) -> dict:
    """Target function that wraps our improved agent for evaluation."""
    thread_id = uuid.uuid4()
    config = {"configurable": {"thread_id": thread_id}}

    result = improved_agent.invoke(
        inputs,
        config=config,
    )
    return {
        "messages": [{"role": "assistant", "content": result["messages"][-1].content}]
    }


# Get the most recent dataset name
most_recent_dataset_name = max(
    client.list_datasets(), key=lambda ds: ds.created_at
).name

# Run evaluation on same dataset
results = client.evaluate(
    improved_target_function,
    data=most_recent_dataset_name,  # Same dataset as section 1
    evaluators=[correctness_evaluator, count_total_tool_calls_evaluator],
    experiment_prefix="with-sql-agent-eval",
    description="Evaluate SQL agent's flexible query generation vs baseline rigid tools",
    max_concurrency=5,
)

## 6. Compare Results in LangSmith

Now let's analyze our improvement using LangSmith's comparison features.